In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1,2])],   
    remainder='passthrough'                         # Leave the rest of the columns untouched
)
X = ct.fit_transform(X)
X = X[:, 1:]
X = np.delete(X,2,1)
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [2]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
# Initialising the ANN
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 7s 854us/step - loss: 0.4077 - acc: 0.8337
Epoch 2/100
8000/8000 [==============================] - 7s 852us/step - loss: 0.4054 - acc: 0.8351
Epoch 3/100
8000/8000 [==============================] - 7s 859us/step - loss: 0.4040 - acc: 0.8325
Epoch 4/100
8000/8000 [==============================] - 7s 929us/step - loss: 0.4023 - acc: 0.8350
Epoch 5/100
8000/8000 [==============================] - 8s 1ms/step - loss: 0.4019 - acc: 0.8341
Epoch 6/100
8000/8000 [==============================] - 8s 965us/step - loss: 0.4008 - acc: 0.8351
Epoch 7/100
8000/8000 [==============================] - 8s 943us/step - loss: 0.3996 - acc: 0.8360
Epoch 8/100
8000/8000 [==============================] - 8s 956us/step - loss: 0.3994 - acc: 0.8367
Epoch 9/100
8000/8000 [==============================] - 8s 955us/step - loss: 0.3985 - acc: 0.8362
Epoch 10/100
8000/8000 [==============================] - 8s 975us/step - loss: 0.3981 - acc: 0.8365
E

Epoch 82/100
8000/8000 [==============================] - 7s 859us/step - loss: 0.3474 - acc: 0.8572
Epoch 83/100
8000/8000 [==============================] - 7s 877us/step - loss: 0.3468 - acc: 0.8609
Epoch 84/100
8000/8000 [==============================] - 7s 858us/step - loss: 0.3457 - acc: 0.8579
Epoch 85/100
8000/8000 [==============================] - 7s 867us/step - loss: 0.3471 - acc: 0.8589
Epoch 86/100
8000/8000 [==============================] - 7s 876us/step - loss: 0.3468 - acc: 0.8580
Epoch 87/100
8000/8000 [==============================] - 7s 865us/step - loss: 0.3465 - acc: 0.8576
Epoch 88/100
8000/8000 [==============================] - 7s 871us/step - loss: 0.3461 - acc: 0.8551
Epoch 89/100
8000/8000 [==============================] - 7s 866us/step - loss: 0.3453 - acc: 0.8596
Epoch 90/100
8000/8000 [==============================] - 7s 859us/step - loss: 0.3452 - acc: 0.8590
Epoch 91/100
8000/8000 [==============================] - 7s 864us/step - loss: 0.3457 - ac

In [7]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(y_pred)
print(cm)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[1507   88]
 [ 188  217]]


In [8]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
print(new_prediction)

[[False]]


In [9]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

print(accuracies)
print(mean)
print(variance)

[0.83875    0.83874999 0.83499999 0.85249999 0.85124999 0.85874999
 0.835      0.82625    0.81125    0.85125   ]
0.8398749949410558
0.0135260156909131


In [10]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [50, 75],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

print ("Grid search values: ")
print(best_parameters)
print(best_accuracy)

Epoch 1/50
6400/6400 [==============================] - 5s 852us/step - loss: 0.5540 - acc: 0.7980
Epoch 2/50
6400/6400 [==============================] - 2s 357us/step - loss: 0.4361 - acc: 0.7980
Epoch 3/50
6400/6400 [==============================] - 2s 355us/step - loss: 0.4302 - acc: 0.7980
Epoch 4/50
6400/6400 [==============================] - 2s 352us/step - loss: 0.4267 - acc: 0.7980
Epoch 5/50
6400/6400 [==============================] - 2s 351us/step - loss: 0.4238 - acc: 0.7980
Epoch 6/50
6400/6400 [==============================] - 2s 350us/step - loss: 0.4208 - acc: 0.7980
Epoch 7/50
6400/6400 [==============================] - 2s 356us/step - loss: 0.4190 - acc: 0.8130
Epoch 8/50
6400/6400 [==============================] - 3s 400us/step - loss: 0.4175 - acc: 0.8214
Epoch 9/50
6400/6400 [==============================] - 2s 375us/step - loss: 0.4166 - acc: 0.8252
Epoch 10/50
6400/6400 [==============================] - 2s 378us/step - loss: 0.4154 - acc: 0.8256
Epoch 11/

6400/6400 [==============================] - 2s 345us/step - loss: 0.4038 - acc: 0.8355
Epoch 34/50
6400/6400 [==============================] - 2s 345us/step - loss: 0.4038 - acc: 0.8362
Epoch 35/50
6400/6400 [==============================] - 2s 349us/step - loss: 0.4036 - acc: 0.8372
Epoch 36/50
6400/6400 [==============================] - 2s 344us/step - loss: 0.4031 - acc: 0.8333
Epoch 37/50
6400/6400 [==============================] - 2s 350us/step - loss: 0.4030 - acc: 0.8366
Epoch 38/50
6400/6400 [==============================] - 2s 345us/step - loss: 0.4032 - acc: 0.8367
Epoch 39/50
6400/6400 [==============================] - 2s 352us/step - loss: 0.4024 - acc: 0.8369
Epoch 40/50
6400/6400 [==============================] - 2s 354us/step - loss: 0.4025 - acc: 0.8348
Epoch 41/50
6400/6400 [==============================] - 2s 364us/step - loss: 0.4023 - acc: 0.8362
Epoch 42/50
6400/6400 [==============================] - 3s 406us/step - loss: 0.4018 - acc: 0.8364
Epoch 43/50


6400/6400 [==============================] - 2s 358us/step - loss: 0.4132 - acc: 0.8270
Epoch 15/50
6400/6400 [==============================] - 2s 373us/step - loss: 0.4126 - acc: 0.8287
Epoch 16/50
6400/6400 [==============================] - 2s 362us/step - loss: 0.4118 - acc: 0.8312
Epoch 17/50
6400/6400 [==============================] - 2s 355us/step - loss: 0.4111 - acc: 0.8314
Epoch 18/50
6400/6400 [==============================] - 2s 360us/step - loss: 0.4106 - acc: 0.8300
Epoch 19/50
6400/6400 [==============================] - 2s 355us/step - loss: 0.4100 - acc: 0.8331
Epoch 20/50
6400/6400 [==============================] - 2s 356us/step - loss: 0.4095 - acc: 0.8311
Epoch 21/50
6400/6400 [==============================] - 2s 356us/step - loss: 0.4091 - acc: 0.8328
Epoch 22/50
6400/6400 [==============================] - 2s 360us/step - loss: 0.4085 - acc: 0.8317
Epoch 23/50
6400/6400 [==============================] - 2s 383us/step - loss: 0.4081 - acc: 0.8339
Epoch 24/50


6400/6400 [==============================] - 2s 362us/step - loss: 0.4018 - acc: 0.8366
Epoch 46/50
6400/6400 [==============================] - 2s 378us/step - loss: 0.4016 - acc: 0.8362
Epoch 47/50
6400/6400 [==============================] - 2s 364us/step - loss: 0.4015 - acc: 0.8356
Epoch 48/50
6400/6400 [==============================] - 2s 368us/step - loss: 0.4017 - acc: 0.8356
Epoch 49/50
6400/6400 [==============================] - 2s 366us/step - loss: 0.4015 - acc: 0.8352
Epoch 50/50
6400/6400 [==============================] - 2s 384us/step - loss: 0.4017 - acc: 0.8367
Epoch 1/50
6400/6400 [==============================] - 3s 416us/step - loss: 0.5764 - acc: 0.7978
Epoch 2/50
6400/6400 [==============================] - 2s 323us/step - loss: 0.4413 - acc: 0.7980
Epoch 3/50
6400/6400 [==============================] - 2s 319us/step - loss: 0.4321 - acc: 0.7980
Epoch 4/50
6400/6400 [==============================] - 2s 330us/step - loss: 0.4285 - acc: 0.7980
Epoch 5/50
6400/

6400/6400 [==============================] - 2s 312us/step - loss: 0.4040 - acc: 0.8372
Epoch 27/50
6400/6400 [==============================] - 2s 312us/step - loss: 0.4040 - acc: 0.8373
Epoch 28/50
6400/6400 [==============================] - 2s 314us/step - loss: 0.4034 - acc: 0.8377
Epoch 29/50
6400/6400 [==============================] - 2s 320us/step - loss: 0.4034 - acc: 0.8369
Epoch 30/50
6400/6400 [==============================] - 2s 343us/step - loss: 0.4026 - acc: 0.8364
Epoch 31/50
6400/6400 [==============================] - 2s 323us/step - loss: 0.4028 - acc: 0.8359
Epoch 32/50
6400/6400 [==============================] - 2s 338us/step - loss: 0.4022 - acc: 0.8372
Epoch 33/50
6400/6400 [==============================] - 2s 358us/step - loss: 0.4022 - acc: 0.8384
Epoch 34/50
6400/6400 [==============================] - 2s 323us/step - loss: 0.4018 - acc: 0.8364
Epoch 35/50
6400/6400 [==============================] - 2s 306us/step - loss: 0.4014 - acc: 0.8359
Epoch 36/50


...

6400/6400 [==============================] - 2s 282us/step - loss: 0.4019 - acc: 0.8359
Epoch 67/75
6400/6400 [==============================] - 2s 278us/step - loss: 0.4021 - acc: 0.8366
Epoch 68/75
6400/6400 [==============================] - 2s 280us/step - loss: 0.4019 - acc: 0.8378
Epoch 69/75
6400/6400 [==============================] - 2s 277us/step - loss: 0.4020 - acc: 0.8355
Epoch 70/75
6400/6400 [==============================] - 2s 295us/step - loss: 0.4014 - acc: 0.8364
Epoch 71/75
6400/6400 [==============================] - 2s 278us/step - loss: 0.4017 - acc: 0.8381
Epoch 72/75
6400/6400 [==============================] - 2s 274us/step - loss: 0.4019 - acc: 0.8364
Epoch 73/75
6400/6400 [==============================] - 2s 277us/step - loss: 0.4014 - acc: 0.8377
Epoch 74/75
6400/6400 [==============================] - 2s 276us/step - loss: 0.4015 - acc: 0.8355
Epoch 75/75
6400/6400 [==============================] - 2s 280us/step - loss: 0.4016 - acc: 0.8370
Epoch 1/75
6

6400/6400 [==============================] - 2s 282us/step - loss: 0.3999 - acc: 0.8361
Epoch 74/75
6400/6400 [==============================] - 2s 283us/step - loss: 0.4000 - acc: 0.8369
Epoch 75/75
6400/6400 [==============================] - 2s 281us/step - loss: 0.3999 - acc: 0.8367
Epoch 1/75
6400/6400 [==============================] - 4s 591us/step - loss: 0.6254 - acc: 0.7908
Epoch 2/75
6400/6400 [==============================] - 2s 285us/step - loss: 0.4784 - acc: 0.7917
Epoch 3/75
6400/6400 [==============================] - 2s 279us/step - loss: 0.4472 - acc: 0.7917
Epoch 4/75
6400/6400 [==============================] - 2s 281us/step - loss: 0.4397 - acc: 0.7917
Epoch 5/75
6400/6400 [==============================] - 2s 290us/step - loss: 0.4357 - acc: 0.7917
Epoch 6/75
6400/6400 [==============================] - 2s 284us/step - loss: 0.4329 - acc: 0.7917
Epoch 7/75
6400/6400 [==============================] - 2s 281us/step - loss: 0.4305 - acc: 0.7917
Epoch 8/75
6400/640

6400/6400 [==============================] - 2s 292us/step - loss: 0.4316 - acc: 0.7973
Epoch 5/75
6400/6400 [==============================] - 2s 273us/step - loss: 0.4283 - acc: 0.7973
Epoch 6/75
6400/6400 [==============================] - 2s 295us/step - loss: 0.4257 - acc: 0.7973
Epoch 7/75
6400/6400 [==============================] - 2s 295us/step - loss: 0.4235 - acc: 0.7973
Epoch 8/75
6400/6400 [==============================] - 4s 596us/step - loss: 0.4213 - acc: 0.7973
Epoch 9/75
6400/6400 [==============================] - 6s 863us/step - loss: 0.4193 - acc: 0.7973 1s - loss:  - ETA: 0s - loss: 0.4213 - a
Epoch 10/75
6400/6400 [==============================] - 6s 923us/step - loss: 0.4176 - acc: 0.8152 4s - loss: - ETA: 1s - loss:
Epoch 11/75
6400/6400 [==============================] - 2s 321us/step - loss: 0.4162 - acc: 0.8223 4s - loss: 0.3794 - acc: 0.82 - ETA:
Epoch 12/75
6400/6400 [==============================] - 2s 279us/step - loss: 0.4153 - acc: 0.8239
Epoch 13/7

6400/6400 [==============================] - 2s 272us/step - loss: 0.4219 - acc: 0.7958
Epoch 10/75
6400/6400 [==============================] - 2s 270us/step - loss: 0.4200 - acc: 0.8114
Epoch 11/75
6400/6400 [==============================] - 2s 279us/step - loss: 0.4181 - acc: 0.8194
Epoch 12/75
6400/6400 [==============================] - ETA: 0s - loss: 0.4174 - acc: 0.826 - 2s 275us/step - loss: 0.4170 - acc: 0.8264
Epoch 13/75
6400/6400 [==============================] - 2s 272us/step - loss: 0.4158 - acc: 0.8273
Epoch 14/75
6400/6400 [==============================] - 2s 270us/step - loss: 0.4148 - acc: 0.8313
Epoch 15/75
6400/6400 [==============================] - 2s 285us/step - loss: 0.4137 - acc: 0.8298
Epoch 16/75
6400/6400 [==============================] - 2s 272us/step - loss: 0.4129 - acc: 0.8314
Epoch 17/75
6400/6400 [==============================] - 2s 276us/step - loss: 0.4123 - acc: 0.8325
Epoch 18/75
6400/6400 [==============================] - 2s 275us/step - l

8000/8000 [==============================] - 6s 778us/step - loss: 0.4147 - acc: 0.8281 6s - loss
Epoch 15/75
8000/8000 [==============================] - 3s 323us/step - loss: 0.4136 - acc: 0.8307
Epoch 16/75
8000/8000 [==============================] - 3s 319us/step - loss: 0.4128 - acc: 0.8317
Epoch 17/75
8000/8000 [==============================] - 3s 313us/step - loss: 0.4120 - acc: 0.8334
Epoch 18/75
8000/8000 [==============================] - 3s 314us/step - loss: 0.4112 - acc: 0.8334
Epoch 19/75
8000/8000 [==============================] - 2s 312us/step - loss: 0.4107 - acc: 0.8322
Epoch 20/75
8000/8000 [==============================] - 3s 318us/step - loss: 0.4101 - acc: 0.8332
Epoch 21/75
8000/8000 [==============================] - 3s 313us/step - loss: 0.4097 - acc: 0.8331
Epoch 22/75
8000/8000 [==============================] - 3s 322us/step - loss: 0.4092 - acc: 0.8346
Epoch 23/75
8000/8000 [==============================] - 3s 313us/step - loss: 0.4086 - acc: 0.8337
Ep